In [1]:
import pandas as pd
from datetime import timedelta, date, datetime


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data
Here is where you can download the data from EPA: https://www.epa.gov/outdoor-air-quality-data/download-daily-data

Select: 
- Pollutant = PM2.5
- Year = 2020 thru 2024 (individual downloads for each, sigh)
- Geographic Area = Houston - The Woodlands - Sugar Land CBSA + All Sites

You can see from the codeblock below that subsequent downloads of past data aren't always identical, telling me that they likely continue to update values long after the dates have passed. So just download files fresh every time you're doing this. I would assume the further back you go, the less likely things are to change but... again... I'm not wasting time on inspecting that now.

## Process
We're looking for monitors that are over 9 and 12 µg/m3 (new and old EPA limits). You take the annual average concentration for each monitor, then you take the average of those annual averages over 3-year periods. If the 3-year avg is over the concentration limit, then we flag the place. Here's [an EPA source](https://experience.arcgis.com/experience/a2ca272ce9fc4019a88ce35b863e2cab#data_s=id%3AdataSource_1-18a8e80642b-layer-19-19147d8da1c-layer-32%3A951) we can use to compare 3-year values for each monitor for fact-checking purposes.

- Remove days known to have high pollution not from industry: NYE, 4th of July
- Group by `Site ID` and `Daily Mean PM2.5 Concentration` and `year` to get annual average concentration for each site, for each year 
- Further group by `Site ID` and `Daily Mean PM2.5 Concentration` and `avgAnnualConc` to calculate 3-year average for timespans 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than current 9 limit
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than past 12 limit
- On your daily data, calculate how many days per year each site had daily averages higher than 35
- Join grouped data to daily overage data

In [2]:
#just checking to make sure the 2023 data didn't change between now and the
#last time I downloaded it.
base_path = '../data/source/EPA/'
file_suffix = '_houmetro_epa_ad_viz_plotval_data'

df2023 = pd.read_csv(f'{base_path}2023{file_suffix}.csv')
df2023v2 = pd.read_csv(f'{base_path}2023v2{file_suffix}.csv')

print(len(df2023))
print(len(df2023v2))

#combine then dedup to see if they are the same
df2023combo = pd.concat([df2023, df2023v2])

print(len(df2023combo))

df2023combo.drop_duplicates(inplace=True)

#ok i don't actually care right now what the differences are. I just need to know
#that there are differences so i'm going to just download all the files i need
#fresh everytime i run this script.
print(len(df2023combo))

4964
4964
9928
4978


In [3]:
base_path = '../data/source/EPA/'
file_suffix = '_houmetro_epa_ad_viz_plotval_data'

years = range(2018, 2025)
pm25_dfs = []
for year in years:
    df = pd.read_csv(f'{base_path}{year}{file_suffix}.csv')
    df['datetime'] = pd.to_datetime(df['Date'])
    df['year'] = df['datetime'].dt.year
    pm25_dfs.append(df)
    
pm25 = pd.concat(pm25_dfs)

In [4]:
print(pm25.dtypes)
print(pm25.datetime.min())
print(pm25.datetime.max())
display(pm25.head())

Date                                      object
Source                                    object
Site ID                                    int64
POC                                        int64
Daily Mean PM2.5 Concentration           float64
Units                                     object
Daily AQI Value                            int64
Local Site Name                           object
Daily Obs Count                            int64
Percent Complete                         float64
AQS Parameter Code                         int64
AQS Parameter Description                 object
Method Code                              float64
Method Description                        object
CBSA Code                                  int64
CBSA Name                                 object
State FIPS Code                            int64
State                                     object
County FIPS Code                           int64
County                                    object
Site Latitude       

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,AQS Parameter Code,AQS Parameter Description,Method Code,Method Description,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude,datetime,year
0,01/02/2018,AQS,481671034,1,4.7,ug/m3 LC,26,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-02,2018
1,01/08/2018,AQS,481671034,1,7.1,ug/m3 LC,39,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-08,2018
2,01/14/2018,AQS,481671034,1,3.4,ug/m3 LC,19,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-14,2018
3,01/20/2018,AQS,481671034,1,4.6,ug/m3 LC,26,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-20,2018
4,01/26/2018,AQS,481671034,1,6.4,ug/m3 LC,36,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-26,2018


- Remove days known to have high pollution not from industry: NYE, 4th of July
- Group by `Site ID` and `Daily Mean PM2.5 Concentration` and `year` to get annual average concentration for each site, for each year 
- Further group by `Site ID` and `Daily Mean PM2.5 Concentration` to calculate 3-year average for timespans 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than current 9 limit
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than past 12 limit
- On your daily data, calculate how many days per year each site had daily averages higher than 35
- Join grouped data to daily overage data

In [5]:
#Remove days known to have high pollution not from industry: NYE, 4th of July
holidays = ['2018-01-01','2018-07-04','2018-12-31',
            '2019-01-01','2019-07-04','2019-12-31',
            '2020-01-01','2020-07-04','2020-12-31',
            '2021-01-01','2021-07-04','2021-12-31',
            '2022-01-01','2022-07-04','2022-12-31',
            '2023-01-01','2023-07-04','2023-12-31',
            '2024-01-01','2024-07-04','2024-12-31']

pm25 = pm25.loc[~pm25['datetime'].isin(holidays)]

#Group by `Site ID` and `year` to get annual average `Daily Mean PM2.5 Concentration` for each site, for each year
by_site = pm25.groupby(['Site ID', 'year'])['Daily Mean PM2.5 Concentration'].mean().reset_index()

#Further group by `Site ID` and `Daily Mean PM2.5 Concentration` to calculate 3-year average for timespans 
# 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
by_yr = pd.pivot_table(by_site, index='Site ID', columns='year', 
                       values='Daily Mean PM2.5 Concentration', aggfunc='mean').reset_index()
by_yr['avg2018_20'] = by_yr[[2018, 2019, 2020]].mean(axis=1,skipna=False)
by_yr['avg2019_21'] = by_yr[[2019, 2020, 2021]].mean(axis=1,skipna=False)
by_yr['avg2020_22'] = by_yr[[2020, 2021, 2022]].mean(axis=1,skipna=False)
by_yr['avg2021_23'] = by_yr[[2021, 2022, 2023]].mean(axis=1,skipna=False)
by_yr['avg2022_24'] = by_yr[[2022, 2023, 2024]].mean(axis=1,skipna=False)

#calculate how many of the past years the site has been higher than current and past limits
def over_limit(x, limit):
    return (x > limit).sum()

def return_max_min(x,limit_type):
    measure = 'max'
    if limit_type == 'max':
        measure = x.max()
    elif limit_type == 'min':
        measure = x.min()
    max_fmt = measure.strftime('%B %Y')
    #print(limit_type,':',max_fmt)
    return max_fmt

over_cols = list(range(2018,2025))
by_yr['yrs_over12'] = by_yr[over_cols].apply(lambda x: over_limit(x,12), axis=1)
by_yr['yrs_over9'] = by_yr[over_cols].apply(lambda x: over_limit(x,9), axis=1)

#calculate how many days per year each site had daily averages higher than 35
daily_over35 = pm25.groupby('Site ID').agg(days_over35=('Daily Mean PM2.5 Concentration', lambda x: over_limit(x, 35)),
                                           max_value=('Daily Mean PM2.5 Concentration', 'max'),
                                           min_value=('Daily Mean PM2.5 Concentration', 'min')
                                        ).reset_index()

daily_over35 = pm25.groupby('Site ID').agg(over35=('Daily Mean PM2.5 Concentration', lambda x: over_limit(x, 35)),
                                                   max_read=('Daily Mean PM2.5 Concentration', 'max'),
                                                   min_read=('Daily Mean PM2.5 Concentration', 'min'),
                                                   min_date=('datetime',lambda x: return_max_min(x,'min')),
                                                   max_date=('datetime',lambda x: return_max_min(x,'max')),
                                                   daily_cnt=('Daily Mean PM2.5 Concentration','count')
                                                   ).reset_index()

pm25_site_summary = by_yr.merge(daily_over35, on='Site ID', how='left')

#lets also join with the sensor features we have
site_info = pm25[['Site ID','County FIPS Code', 'County', 'Site Latitude', 'Site Longitude']].drop_duplicates()
pm25_site_summary = pm25_site_summary.merge(site_info, on='Site ID', how='left')
pm25_site_summary.columns = pm25_site_summary.columns.astype(str)

display(pm25_site_summary.columns)

#if the 2024 values is greater than 9, list "out" for compliance else "in"
pm25_site_summary['compliance_2024'] = pm25_site_summary['2024'].apply(lambda x: 'out' if x > 9 else 'in')

#renaming some columns because i'm anal
rename_cols = {'Site ID':'site_id','County FIPS Code':'cnty_fips','County':'county',
               'Site Latitude':'latitude','Site Longitude':'longitude'}
pm25_site_summary.rename(columns=rename_cols,inplace=True)

#export for graphics and whatnot
pm25_site_summary['monitor_type'] = 'EPA'
pm25_site_summary.rename(columns={'2018':'avg_2018',
                                  '2019':'avg_2019',
                                  '2020':'avg_2020',
                                  '2021':'avg_2021',
                                  '2022':'avg_2022',
                                  '2023':'avg_2023',
                                  '2024':'avg_2024'},inplace=True)

reorder_cols = ['longitude','latitude','site_id', 'avg_2022', 'avg_2023', 'avg_2024', 
                'avg2022_24', 'yrs_over12','yrs_over9', 'over35', 'max_read', 'min_read', 
                'cnty_fips','county','monitor_type','compliance_2024','min_date',
                'max_date','daily_cnt']
pm25_site_summary[reorder_cols].to_csv('../data/analyzed/houmetro-epa-pm25-site-summary.csv',index=False)

/var/folders/2g/mqj0d6v95vbfcqgf7kpgrsrc0000gr/T/ipykernel_3927/3434297536.py:10: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  pm25 = pm25.loc[~pm25['datetime'].isin(holidays)]


Index(['Site ID', '2018', '2019', '2020', '2021', '2022', '2023', '2024',
       'avg2018_20', 'avg2019_21', 'avg2020_22', 'avg2021_23', 'avg2022_24',
       'yrs_over12', 'yrs_over9', 'over35', 'max_read', 'min_read', 'min_date',
       'max_date', 'daily_cnt', 'County FIPS Code', 'County', 'Site Latitude',
       'Site Longitude'],
      dtype='object')

In [ ]:
print(pm25_site_summary.columns)
display(pm25_site_summary.head())

In [ ]:
#482010046 is Settegast
pm25.loc[pm25['Site ID'] == 482010046].sort_values('datetime')

### Is an average of averages the same as the average of all of the raw values? Let's test


In [27]:
#get some example site_ids
display(pm25.groupby(['Site ID']).size().reset_index(name='counts').sort_values('counts', ascending=False).head())
display(pm25.groupby(['Site ID']).size().reset_index(name='counts').sort_values('counts', ascending=False).sample(5))


,Site ID,counts
8,482011035,5255
9,482011039,3925
2,482010024,2700
5,482010058,2430
7,482011034,2408


,Site ID,counts
3,482010046,1248
10,482011050,2357
6,482010066,1281
1,481671034,2301
2,482010024,2700


In [44]:
def compare_avg_types(example_id):
    ex_site = pm25.loc[pm25['Site ID'] == example_id]
    ex_site_18_20 = ex_site.loc[(ex_site['year'] >= 2018) & (ex_site['year'] <= 2020)]
    
    value_mean = ex_site_18_20['Daily Mean PM2.5 Concentration'].mean()
    avg_avg = pm25_site_summary.loc[pm25_site_summary['site_id'] == example_id]['avg2018_20'].values[0]
    
    print(round(value_mean,2),'vs',round(avg_avg,2))

In [48]:
compare_avg_types(482011035)
compare_avg_types(482011039)
compare_avg_types(482011052)
compare_avg_types(481671034)
compare_avg_types(482010024)
compare_avg_types(482011050)

10.11 vs 10.11
8.07 vs 8.16
10.42 vs 10.41
6.72 vs 6.83
9.4 vs 9.39
7.33 vs 7.3
